<a href="https://colab.research.google.com/github/marcelogarret/reconpad/blob/main/aula5a_gridsearch_garret_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Importando o dataset
from seaborn import load_dataset
import pandas as pd
df = load_dataset('penguins')
df = df.dropna()

In [ ]:
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male


In [ ]:
X, y = df[df.columns[0:(len(df.columns)-1)]].to_numpy(), df[df.columns[(len(df.columns)-1)]].replace({'Male':1, 'Female':0}).to_numpy()
X.shape, y.shape

((333, 6), (333,))

In [ ]:
## Importando bibliotecas de seleção de modelos, métricas, pré-processamento e pipeline
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_validate, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
import numpy as np

## Configuração 1 do Pipeline ##
## Construindo o Pipeline com o pré-processamento e o modelo de regressão Knn
pipe = Pipeline([
    ('pre', OneHotEncoder(handle_unknown='ignore')),
    ('clf', KNeighborsClassifier())
])

## Definindo parãmetros de ajuste
param = {'pre__sparse_output': [True, False],
         'pre__min_frequency': [50, 100],
         'clf__n_neighbors': [1, 3, 5]}

## Definindo modelo aninhado com Pipeline interno ao GridSearchcV
modeloAninhado = GridSearchCV(pipe, param, verbose=0, cv=5,
                      scoring=make_scorer(mean_squared_error,
                                          greater_is_better=False,
                                          squared=False))

## Realizando a validação cruzada com 5 folds e retornando o resultado do treinamento
scores = cross_validate(modeloAninhado, X, y, cv=5, return_estimator=True,
                        scoring=make_scorer(mean_squared_error, squared=False))

## Comparando o resultado médio dos treinamentos e dos testes da validação cruzada
print(f"RMSE = {np.mean(scores['test_score'])}")

RMSE = 0.7227601452261643


In [ ]:
## Configuração 2 do Pipeline ##

## Definindo parãmetros de ajuste
param2 = {'n_neighbors': [1, 3, 5]}

## Definindo modelo de GridSearach com KNN
GridSearchKNN = GridSearchCV(KNeighborsClassifier(), param2, verbose=0, cv=5,
                      scoring=make_scorer(mean_squared_error,
                                          greater_is_better=False,
                                          squared=False))

## Construindo o Pipeline com o pré-processamento e o modelo de GridSearchKNN
modeloAninhado2 = Pipeline([
    ('pre', OneHotEncoder(handle_unknown='ignore')),
    ('clf', GridSearchKNN)
])

## Realizando a validação cruzada com 5 folds e retornando o resultado do treinamento
scores = cross_validate(modeloAninhado2, X, y, cv=5, return_estimator=True,
                        scoring=make_scorer(mean_squared_error, squared=False), return_train_score=True)

## Comparando o resultado médio dos treinamentos e dos testes da validação cruzada
print(f"RMSE = {np.mean(scores['test_score'])}")

RMSE = 0.5617532198709724
